In [1]:
import re
import nltk 
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def load_data():
    data = pd.read_csv('./full-corpus.csv')
    return data

In [3]:
tweet_df = load_data()
tweet_df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [4]:
print('Dataset size:',tweet_df.shape)
print('Columns are:',tweet_df.columns)

Dataset size: (5113, 5)
Columns are: Index(['Topic', 'Sentiment', 'TweetId', 'TweetDate', 'TweetText'], dtype='object')


In [5]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5113 entries, 0 to 5112
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Topic      5113 non-null   object
 1   Sentiment  5113 non-null   object
 2   TweetId    5113 non-null   int64 
 3   TweetDate  5113 non-null   object
 4   TweetText  5113 non-null   object
dtypes: int64(1), object(4)
memory usage: 199.9+ KB


In [22]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
    text = re.sub("(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
    return text

tweet_df['Tweet_punct'] = tweet_df['TweetText'].apply(lambda x: remove_punct(x))
tweet_df.head(10)

,Topic,Sentiment,TweetId,TweetDate,TweetText,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,Now all Apple has to do is get swype on the ip...,"[now, all, apple, has, to, do, is, get, swype,...","[apple, get, swype, iphone, crack, iphone]","[appl, get, swype, iphon, crack, iphon]","[apple, get, swype, iphone, crack, iphone]"
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,Apple will be adding more carrier support to t...,"[apple, will, be, adding, more, carrier, suppo...","[apple, adding, carrier, support, iphone, anno...","[appl, ad, carrier, support, iphon, announc]","[apple, adding, carrier, support, iphone, anno..."
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,Hilarious youtube video guy does a duet with ...,"[hilarious, youtube, video, guy, does, a, duet...","[hilarious, youtube, video, guy, duet, apple, ...","[hilari, youtub, video, guy, duet, appl, siri,...","[hilarious, youtube, video, guy, duet, apple, ..."
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,RIM you made it too easy for me to switch to A...,"[rim, you, made, it, too, easy, for, me, to, s...","[rim, made, easy, switch, apple, iphone, see, ya]","[rim, made, easi, switch, appl, iphon, see, ya]","[rim, made, easy, switch, apple, iphone, see, ya]"
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...,"[i, just, realized, that, the, reason, i, got,...","[realized, reason, got, twitter, ios, thanks, ...","[realiz, reason, got, twitter, io, thank, appl]","[realized, reason, got, twitter, io, thanks, a..."
5,apple,positive,126394830791254016,Tue Oct 18 20:30:50 +0000 2011,"I'm a current @Blackberry user, little bit dis...",Im a current Blackberry user little bit disapp...,"[im, a, current, blackberry, user, little, bit...","[im, current, blackberry, user, little, bit, d...","[im, current, blackberri, user, littl, bit, di...","[im, current, blackberry, user, little, bit, d..."
6,apple,positive,126379685453119488,Tue Oct 18 19:30:39 +0000 2011,The 16 strangest things Siri has said so far. ...,The strangest things Siri has said so far I a...,"[the, strangest, things, siri, has, said, so, ...","[strangest, things, siri, said, far, sooo, gla...","[strangest, thing, siri, said, far, sooo, glad...","[strangest, thing, siri, said, far, sooo, glad..."
7,apple,positive,126377656416612353,Tue Oct 18 19:22:35 +0000 2011,Great up close & personal event @Apple tonight...,Great up close personal event Apple tonight i...,"[great, up, close, personal, event, apple, ton...","[great, close, personal, event, apple, tonight...","[great, close, person, event, appl, tonight, r...","[great, close, personal, event, apple, tonight..."
8,apple,positive,126373779483004928,Tue Oct 18 19:07:11 +0000 2011,From which companies do you experience the bes...,From which companies do you experience the bes...,"[from, which, companies, do, you, experience, ...","[companies, experience, best, customer, servic...","[compani, experi, best, custom, servic, asid, ...","[company, experience, best, customer, service,..."
9,apple,positive,126366353757179904,Tue Oct 18 18:37:41 +0000 2011,"Just apply for a job at @Apple, hope they call...",Just apply for a job at Apple hope they call m...,"[just, apply, for, a, job, at, apple, hope, th...","[apply, job, apple, hope, call, lol]","[appli, job, appl, hope, call, lol]","[apply, job, apple, hope, call, lol]"


In [7]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

tweet_df['Tweet_tokenized'] = tweet_df['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
tweet_df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Tweet_punct,Tweet_tokenized
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,Now all Apple has to do is get swype on the ip...,"[now, all, apple, has, to, do, is, get, swype,..."
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,Apple will be adding more carrier support to t...,"[apple, will, be, adding, more, carrier, suppo..."
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,Hilarious youtube video guy does a duet with a...,"[hilarious, youtube, video, guy, does, a, duet..."
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,RIM you made it too easy for me to switch to A...,"[rim, you, made, it, too, easy, for, me, to, s..."
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...,"[i, just, realized, that, the, reason, i, got,..."


In [8]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
tweet_df['Tweet_nonstop'] = tweet_df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweet_df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Tweet_punct,Tweet_tokenized,Tweet_nonstop
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,Now all Apple has to do is get swype on the ip...,"[now, all, apple, has, to, do, is, get, swype,...","[apple, get, swype, iphone, crack, iphone]"
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,Apple will be adding more carrier support to t...,"[apple, will, be, adding, more, carrier, suppo...","[apple, adding, carrier, support, iphone, anno..."
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,Hilarious youtube video guy does a duet with a...,"[hilarious, youtube, video, guy, does, a, duet...","[hilarious, youtube, video, guy, duet, apple, ..."
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,RIM you made it too easy for me to switch to A...,"[rim, you, made, it, too, easy, for, me, to, s...","[rim, made, easy, switch, apple, iphone, see, ya]"
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...,"[i, just, realized, that, the, reason, i, got,...","[realized, reason, got, twitter, ios, thanks, ..."


In [9]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tweet_df['Tweet_stemmed'] = tweet_df['Tweet_nonstop'].apply(lambda x: stemming(x))
tweet_df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,Now all Apple has to do is get swype on the ip...,"[now, all, apple, has, to, do, is, get, swype,...","[apple, get, swype, iphone, crack, iphone]","[appl, get, swype, iphon, crack, iphon]"
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,Apple will be adding more carrier support to t...,"[apple, will, be, adding, more, carrier, suppo...","[apple, adding, carrier, support, iphone, anno...","[appl, ad, carrier, support, iphon, announc]"
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,Hilarious youtube video guy does a duet with a...,"[hilarious, youtube, video, guy, does, a, duet...","[hilarious, youtube, video, guy, duet, apple, ...","[hilari, youtub, video, guy, duet, appl, siri,..."
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,RIM you made it too easy for me to switch to A...,"[rim, you, made, it, too, easy, for, me, to, s...","[rim, made, easy, switch, apple, iphone, see, ya]","[rim, made, easi, switch, appl, iphon, see, ya]"
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...,"[i, just, realized, that, the, reason, i, got,...","[realized, reason, got, twitter, ios, thanks, ...","[realiz, reason, got, twitter, io, thank, appl]"


In [10]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

tweet_df['Tweet_lemmatized'] = tweet_df['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
tweet_df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,Tweet_punct,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,Now all Apple has to do is get swype on the ip...,"[now, all, apple, has, to, do, is, get, swype,...","[apple, get, swype, iphone, crack, iphone]","[appl, get, swype, iphon, crack, iphon]","[apple, get, swype, iphone, crack, iphone]"
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,Apple will be adding more carrier support to t...,"[apple, will, be, adding, more, carrier, suppo...","[apple, adding, carrier, support, iphone, anno...","[appl, ad, carrier, support, iphon, announc]","[apple, adding, carrier, support, iphone, anno..."
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,Hilarious youtube video guy does a duet with a...,"[hilarious, youtube, video, guy, does, a, duet...","[hilarious, youtube, video, guy, duet, apple, ...","[hilari, youtub, video, guy, duet, appl, siri,...","[hilarious, youtube, video, guy, duet, apple, ..."
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,RIM you made it too easy for me to switch to A...,"[rim, you, made, it, too, easy, for, me, to, s...","[rim, made, easy, switch, apple, iphone, see, ya]","[rim, made, easi, switch, appl, iphon, see, ya]","[rim, made, easy, switch, apple, iphone, see, ya]"
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...,"[i, just, realized, that, the, reason, i, got,...","[realized, reason, got, twitter, ios, thanks, ...","[realiz, reason, got, twitter, io, thank, appl]","[realized, reason, got, twitter, io, thanks, a..."


In [11]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation])
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc) 
    text = [ps.stem(word) for word in tokens if word not in stopword]
    return text

In [12]:
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(tweet_df['TweetText'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

5113 Number of tweets has 13792 words


In [23]:
countVector.head()

AttributeError: head not found

In [20]:
x="jonh @jbjb gigj"
x = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
x

'jonh gigj'